<a href="https://colab.research.google.com/github/veleronie/NetworkFlowAnomalyDetection/blob/main/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import pandas as pd
import sklearn
import pickle

path = os.getcwd()
path_to_datasets = r'/drive/MyDrive/AnomalyDetection/IDS2017/TrafficLabelling'
all_files = os.listdir(path + path_to_datasets)    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
csv_files

['Monday-WorkingHours.pcap_ISCX.csv',
 'Wednesday-workingHours.pcap_ISCX.csv',
 'Tuesday-WorkingHours.pcap_ISCX.csv',
 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
 'Friday-WorkingHours-Morning.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv']

The aim of this file is to make the csv files clean from any errors: drop missing/meaningless values, handling specific character encodings, etc.

In [ ]:
test_file = pd.read_csv('/content/drive/MyDrive/AnomalyDetection/IDS2017/TrafficLabelling/Wednesday-workingHours.pcap_ISCX.csv')
with open('pickled_wednesday_data.pkl', 'wb') as f:
  pickle.dump(test_file, f)
#test_file = pd.read_csv('/content/drive/MyDrive/AnomalyDetection/IDS2017/TrafficLabelling/Wednesday-workingHours.pcap_ISCX.csv')
with open('pickled_wednesday_data.pkl', 'rb') as f:
  test_file = pickle.load(f)
test_file.head()


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.14-209.48.71.168-49459-80-6,192.168.10.14,49459,209.48.71.168,80,6,5/7/2017 8:42,38308,1,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.3-192.168.10.17-389-49453-6,192.168.10.17,49453,192.168.10.3,389,6,5/7/2017 8:42,479,11,5,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.3-192.168.10.17-88-46124-6,192.168.10.17,46124,192.168.10.3,88,6,5/7/2017 8:42,1095,10,6,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.3-192.168.10.17-389-49454-6,192.168.10.17,49454,192.168.10.3,389,6,5/7/2017 8:42,15206,17,12,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.17-88-46126-6,192.168.10.17,46126,192.168.10.3,88,6,5/7/2017 8:42,1092,9,6,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [ ]:
import numpy as np
main_labels = list(test_file.columns)
labels_with_missing_values = []
for label in main_labels:
  if test_file[label].isnull().sum() > 0:
    labels_with_missing_values.append(label)
labels_with_missing_values

['Flow Bytes/s']

In [ ]:
test_file[labels_with_missing_values[0]].fillna(0, inplace = True)
numeric_columns = test_file.select_dtypes(include=np.number).columns
#test_file[pd.to_numeric(test_file[' Flow Packets/s'], errors='coerce').isnull()][' Flow Packets/s'].unique()

In [ ]:
#test_file['Flow Bytes/s'].replace([np.inf, -np.inf], -1, inplace=True)
#test_file['Flow Bytes/s'].max()

In [ ]:
for column in numeric_columns:
  if np.isin([-np.inf, np.inf], test_file['Flow Bytes/s']).any():
      test_file[column].replace([np.inf, -np.inf], -1, inplace=True)  #replacing inf values with -1


#is_inf = np.isin([-np.inf, np.inf], test_file['Flow Bytes/s'])
#count = np.isinf(test_file['Flow Bytes/s']).values.sum()